For Question #2, filter to CMS users (cmslocal, cmspilot), examine relationship betw early (< 60 min) termination jobs, production & nogpfs partitions, and specific nodes.  Look for commonly failed nodes

Actions: 1) pull in code to convert time to seconds 2) run query for cms users, used time less than 3600 seconds and 1800 seconds, partition = production, nogpfs. 3) analyze exit codes: left = user errors, right = node errors. 4) run query for cms user, 30 mins, plus State = Failed.  4) analyze for commonly occurring failed nodes

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import scipy.stats as stats
import statsmodels.api as sm
%matplotlib inline
from io import StringIO
import re

In [2]:
for_pd = StringIO()
with open('../data/accre-jobs-2020.csv') as accre:
    for line in accre:
        new_line = re.sub(r',', '|', line.rstrip(), count=12)
        print (new_line, file=for_pd)

for_pd.seek(0)

accre = pd.read_csv(for_pd, sep='|')
print (accre)

               JOBID        ACCOUNT       USER    REQMEM    USEDMEM  \
0           15925210        treviso   arabella  122880Mn  65973.49M   
1           15861126        treviso   arabella  122880Mn  67181.12M   
2           15861125        treviso   arabella  122880Mn  69111.86M   
3           16251645        treviso   arabella  122880Mn  65317.33M   
4           16251646        treviso   arabella  122880Mn  65876.11M   
...              ...            ...        ...       ...        ...   
3816285   24173815_8     portabella     vennie   32768Mn  30042.68M   
3816286   24173815_9     portabella     vennie   32768Mn  31067.75M   
3816287  24173815_10     portabella     vennie   32768Mn  21207.96M   
3816288     24173817  boysenberries  hortensia  100000Mn          0   
3816289     24173866    horseradish     ariane   20480Mn          0   

             REQTIME     USEDTIME  NODES  CPUS   PARTITION EXITCODE  \
0        13-18:00:00  13-18:00:28      1    24  production      0:0   
1    

In [3]:
accre.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3816290 entries, 0 to 3816289
Data columns (total 13 columns):
 #   Column     Dtype 
---  ------     ----- 
 0   JOBID      object
 1   ACCOUNT    object
 2   USER       object
 3   REQMEM     object
 4   USEDMEM    object
 5   REQTIME    object
 6   USEDTIME   object
 7   NODES      int64 
 8   CPUS       int64 
 9   PARTITION  object
 10  EXITCODE   object
 11  STATE      object
 12  NODELIST   object
dtypes: int64(2), object(11)
memory usage: 378.5+ MB


In [4]:
# bring in code to convert time to seconds 
# Create a function to split the hh:mm:ss string and calculate seconds from it
def to_sec(x):
    h,m,s = map(int,x.split(':'))
    return (h*60+m)*60+s

In [5]:
accre['REQTIME_DAY_SEC'] = accre['REQTIME'].str.extract('(\d+)-')
accre['REQTIME_DAY_SEC'] = pd.to_numeric(accre['REQTIME_DAY_SEC'])
accre['REQTIME_DAY_SEC'] = accre['REQTIME_DAY_SEC'].fillna(0)
accre['REQTIME_DAY_SEC'] = accre['REQTIME_DAY_SEC']*24*60*60
# Extract the hh:mm:ss from REQTIME, put in a new column, and then apply the to_sec function 
accre['REQTIME_T'] = accre['REQTIME'].str.extract('(..:..:..)$')
# REQTIME_SEC includes total seconds from REQTIME
accre['REQTIME_SEC'] = accre['REQTIME_T'].apply(to_sec) + accre['REQTIME_DAY_SEC']
# Do the same for USEDTIME
accre['USEDTIME_DAY_SEC'] = accre['USEDTIME'].str.extract('(\d+)-')
accre['USEDTIME_DAY_SEC'] = pd.to_numeric(accre['USEDTIME_DAY_SEC'])
accre['USEDTIME_DAY_SEC'] = accre['USEDTIME_DAY_SEC'].fillna(0)
accre['USEDTIME_DAY_SEC'] = accre['USEDTIME_DAY_SEC']*24*60*60
# Do the same for USEDTIME
accre['USEDTIME_T'] = accre['USEDTIME'].str.extract('(..:..:..)$')
# USEDTIME_SEC includes total second from USEDTIME
accre['USEDTIME_SEC'] = accre['USEDTIME_T'].apply(to_sec) + accre['USEDTIME_DAY_SEC']
# Check to make sure the data types look okay
accre.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3816290 entries, 0 to 3816289
Data columns (total 19 columns):
 #   Column            Dtype  
---  ------            -----  
 0   JOBID             object 
 1   ACCOUNT           object 
 2   USER              object 
 3   REQMEM            object 
 4   USEDMEM           object 
 5   REQTIME           object 
 6   USEDTIME          object 
 7   NODES             int64  
 8   CPUS              int64  
 9   PARTITION         object 
 10  EXITCODE          object 
 11  STATE             object 
 12  NODELIST          object 
 13  REQTIME_DAY_SEC   float64
 14  REQTIME_T         object 
 15  REQTIME_SEC       float64
 16  USEDTIME_DAY_SEC  float64
 17  USEDTIME_T        object 
 18  USEDTIME_SEC      float64
dtypes: float64(4), int64(2), object(13)
memory usage: 553.2+ MB


In [18]:
accre_cms = accre.drop(['REQMEM', 'USEDMEM', 'REQTIME', 'USEDTIME', 'REQTIME_DAY_SEC', 'REQTIME_T', 'USEDTIME_DAY_SEC', 'USEDTIME_T'], axis = 1)

In [19]:
accre_cms.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3816290 entries, 0 to 3816289
Data columns (total 11 columns):
 #   Column        Dtype  
---  ------        -----  
 0   JOBID         object 
 1   ACCOUNT       object 
 2   USER          object 
 3   NODES         int64  
 4   CPUS          int64  
 5   PARTITION     object 
 6   EXITCODE      object 
 7   STATE         object 
 8   NODELIST      object 
 9   REQTIME_SEC   float64
 10  USEDTIME_SEC  float64
dtypes: float64(2), int64(2), object(7)
memory usage: 320.3+ MB


First, filter the users to cmspilot and cmslocal

In [20]:
accre_cms_user = accre_cms.loc[accre_cms['USER'].isin(['cmslocal', 'cmspilot'])]
accre_cms_user

,JOBID,ACCOUNT,USER,NODES,CPUS,PARTITION,EXITCODE,STATE,NODELIST,REQTIME_SEC,USEDTIME_SEC
2125,17032244,cms,cmspilot,1,4,production,0:0,COMPLETED,cn1205,172800.0,171106.0
2126,17032568,cms,cmspilot,1,4,production,0:0,COMPLETED,cn1492,172800.0,171193.0
2127,17032569,cms,cmspilot,1,4,production,0:0,COMPLETED,cn1497,172800.0,171396.0
2128,17032916,cms,cmspilot,1,8,nogpfs,0:0,COMPLETED,ng1037,172800.0,170875.0
2129,17032920,cms,cmspilot,1,8,nogpfs,0:0,COMPLETED,ng691,172800.0,170164.0
...,...,...,...,...,...,...,...,...,...,...,...
3815978,24173154,cms,cmspilot,1,4,production,0:0,RUNNING,cn461,172800.0,401.0
3815979,24173155,cms,cmspilot,1,4,production,0:0,RUNNING,cn1516,172800.0,244.0
3815980,24173156,cms,cmspilot,1,4,production,0:0,RUNNING,cn1524,172800.0,244.0
3815981,24173158,cms,cmspilot,1,4,production,0:0,RUNNING,cn1532,172800.0,244.0


In [21]:
accre_cms_user.USER.unique()

array(['cmspilot', 'cmslocal'], dtype=object)

In [22]:
accre_cms_user.PARTITION.unique()

array(['production', 'nogpfs', 'pascal'], dtype=object)

In [23]:
accre_cms_user.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 836628 entries, 2125 to 3816000
Data columns (total 11 columns):
 #   Column        Non-Null Count   Dtype  
---  ------        --------------   -----  
 0   JOBID         836628 non-null  object 
 1   ACCOUNT       836628 non-null  object 
 2   USER          836628 non-null  object 
 3   NODES         836628 non-null  int64  
 4   CPUS          836628 non-null  int64  
 5   PARTITION     836628 non-null  object 
 6   EXITCODE      836628 non-null  object 
 7   STATE         836628 non-null  object 
 8   NODELIST      836628 non-null  object 
 9   REQTIME_SEC   836628 non-null  float64
 10  USEDTIME_SEC  836628 non-null  float64
dtypes: float64(2), int64(2), object(7)
memory usage: 76.6+ MB


In [24]:
accre_cms_user.ACCOUNT.unique()

array(['cms', 'cms_lowprio', 'cms_gpu_acc'], dtype=object)

Note that although filtered to cmslocal and cmspilot, there are still 3 Accounts 

Now filter the partitions to production and nogpfs

In [25]:
accre_cms_user_part = accre_cms_user.loc[accre_cms_user['PARTITION'].isin(['production', 'nogpfs'])]
accre_cms_user_part

,JOBID,ACCOUNT,USER,NODES,CPUS,PARTITION,EXITCODE,STATE,NODELIST,REQTIME_SEC,USEDTIME_SEC
2125,17032244,cms,cmspilot,1,4,production,0:0,COMPLETED,cn1205,172800.0,171106.0
2126,17032568,cms,cmspilot,1,4,production,0:0,COMPLETED,cn1492,172800.0,171193.0
2127,17032569,cms,cmspilot,1,4,production,0:0,COMPLETED,cn1497,172800.0,171396.0
2128,17032916,cms,cmspilot,1,8,nogpfs,0:0,COMPLETED,ng1037,172800.0,170875.0
2129,17032920,cms,cmspilot,1,8,nogpfs,0:0,COMPLETED,ng691,172800.0,170164.0
...,...,...,...,...,...,...,...,...,...,...,...
3815978,24173154,cms,cmspilot,1,4,production,0:0,RUNNING,cn461,172800.0,401.0
3815979,24173155,cms,cmspilot,1,4,production,0:0,RUNNING,cn1516,172800.0,244.0
3815980,24173156,cms,cmspilot,1,4,production,0:0,RUNNING,cn1524,172800.0,244.0
3815981,24173158,cms,cmspilot,1,4,production,0:0,RUNNING,cn1532,172800.0,244.0


In [26]:
accre_cms_user_part.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 835473 entries, 2125 to 3816000
Data columns (total 11 columns):
 #   Column        Non-Null Count   Dtype  
---  ------        --------------   -----  
 0   JOBID         835473 non-null  object 
 1   ACCOUNT       835473 non-null  object 
 2   USER          835473 non-null  object 
 3   NODES         835473 non-null  int64  
 4   CPUS          835473 non-null  int64  
 5   PARTITION     835473 non-null  object 
 6   EXITCODE      835473 non-null  object 
 7   STATE         835473 non-null  object 
 8   NODELIST      835473 non-null  object 
 9   REQTIME_SEC   835473 non-null  float64
 10  USEDTIME_SEC  835473 non-null  float64
dtypes: float64(2), int64(2), object(7)
memory usage: 76.5+ MB


In [27]:
accre_cms_user_part.PARTITION.unique()

array(['production', 'nogpfs'], dtype=object)

In [111]:
accre_cms_user_part.ACCOUNT.unique()

array(['cms', 'cms_lowprio'], dtype=object)

Note there are now 2 Accounts associated with cmslocal and cmspilot

So we now have a database with cmslocal/cmspilot as the Users and production/nogpfs as the Partitions. Next, filter to jobs less than 60 minutes, and a separate df with 30 minutes.  (Note: I originally tried to first filter on original USEDTIME less than 1:00 but got error '<' not supported betw 'list' and 'str'

In [97]:
# df with jobs less than 60 minutes.  I used 60 minutes, 2 seconds in case the last digit is not included and 
#there's a delay in cancel execution.  
accre_cms_user_part_1hr = accre_cms_user_part[accre_cms_user_part['USEDTIME_SEC'].between(0,3602)]
accre_cms_user_part_1hr

,JOBID,ACCOUNT,USER,NODES,CPUS,PARTITION,EXITCODE,STATE,NODELIST,REQTIME_SEC,USEDTIME_SEC
8061,17093431,cms,cmslocal,1,8,nogpfs,0:0,COMPLETED,ng1017,172800.0,1327.0
8062,17093435,cms,cmslocal,1,8,nogpfs,0:0,COMPLETED,ng1027,172800.0,1311.0
8064,17093439,cms,cmslocal,1,8,nogpfs,0:0,COMPLETED,ng1020,172800.0,1224.0
8065,17093440,cms,cmslocal,1,8,nogpfs,0:0,COMPLETED,ng1036,172800.0,1224.0
8066,17093441,cms,cmslocal,1,8,nogpfs,0:0,COMPLETED,ng1003,172800.0,1231.0
...,...,...,...,...,...,...,...,...,...,...,...
3815978,24173154,cms,cmspilot,1,4,production,0:0,RUNNING,cn461,172800.0,401.0
3815979,24173155,cms,cmspilot,1,4,production,0:0,RUNNING,cn1516,172800.0,244.0
3815980,24173156,cms,cmspilot,1,4,production,0:0,RUNNING,cn1524,172800.0,244.0
3815981,24173158,cms,cmspilot,1,4,production,0:0,RUNNING,cn1532,172800.0,244.0


In [98]:
accre_cms_user_part_1hr.describe()

,NODES,CPUS,REQTIME_SEC,USEDTIME_SEC
count,509499.0,509499.000000,509499.000000,509499.000000
mean,1.0,6.622384,170990.814506,961.520470
std,0.0,6.154231,17464.355821,693.418003
min,1.0,0.000000,1800.000000,0.000000
25%,1.0,4.000000,172800.000000,148.000000
50%,1.0,4.000000,172800.000000,1207.000000
75%,1.0,8.000000,172800.000000,1239.000000
max,1.0,64.000000,172800.000000,3602.000000


In [112]:
#df with jobs less than 30 minutes.  I used 30 minutes, 2 seconds in case the last digit is not included and 
#there's a delay in cancel execution. 
accre_cms_user_part_30 = accre_cms_user_part[accre_cms_user_part['USEDTIME_SEC'].between(0,1802)]
accre_cms_user_part_30

,JOBID,ACCOUNT,USER,NODES,CPUS,PARTITION,EXITCODE,STATE,NODELIST,REQTIME_SEC,USEDTIME_SEC
8061,17093431,cms,cmslocal,1,8,nogpfs,0:0,COMPLETED,ng1017,172800.0,1327.0
8062,17093435,cms,cmslocal,1,8,nogpfs,0:0,COMPLETED,ng1027,172800.0,1311.0
8064,17093439,cms,cmslocal,1,8,nogpfs,0:0,COMPLETED,ng1020,172800.0,1224.0
8065,17093440,cms,cmslocal,1,8,nogpfs,0:0,COMPLETED,ng1036,172800.0,1224.0
8066,17093441,cms,cmslocal,1,8,nogpfs,0:0,COMPLETED,ng1003,172800.0,1231.0
...,...,...,...,...,...,...,...,...,...,...,...
3815978,24173154,cms,cmspilot,1,4,production,0:0,RUNNING,cn461,172800.0,401.0
3815979,24173155,cms,cmspilot,1,4,production,0:0,RUNNING,cn1516,172800.0,244.0
3815980,24173156,cms,cmspilot,1,4,production,0:0,RUNNING,cn1524,172800.0,244.0
3815981,24173158,cms,cmspilot,1,4,production,0:0,RUNNING,cn1532,172800.0,244.0


In [95]:
accre_cms_user_part_30.describe()

,NODES,CPUS,REQTIME_SEC,USEDTIME_SEC
count,474044.0,474044.000000,474044.000000,474044.000000
mean,1.0,6.823154,172034.455873,844.716611
std,0.0,6.231422,11370.900026,546.847968
min,1.0,0.000000,1800.000000,0.000000
25%,1.0,4.000000,172800.000000,110.000000
50%,1.0,8.000000,172800.000000,1205.000000
75%,1.0,8.000000,172800.000000,1230.000000
max,1.0,64.000000,172800.000000,1802.000000


474044 cms jobs, or 57%, are less than 30 minutes.  509,499 cms jobs, or 61%, are less than 1 hr 

In [96]:
accre_cms_user_part_30.sort_values('USEDTIME_SEC',ascending = 'false')
accre_cms_user_part_30

,JOBID,ACCOUNT,USER,NODES,CPUS,PARTITION,EXITCODE,STATE,NODELIST,REQTIME_SEC,USEDTIME_SEC
8061,17093431,cms,cmslocal,1,8,nogpfs,0:0,COMPLETED,ng1017,172800.0,1327.0
8062,17093435,cms,cmslocal,1,8,nogpfs,0:0,COMPLETED,ng1027,172800.0,1311.0
8064,17093439,cms,cmslocal,1,8,nogpfs,0:0,COMPLETED,ng1020,172800.0,1224.0
8065,17093440,cms,cmslocal,1,8,nogpfs,0:0,COMPLETED,ng1036,172800.0,1224.0
8066,17093441,cms,cmslocal,1,8,nogpfs,0:0,COMPLETED,ng1003,172800.0,1231.0
...,...,...,...,...,...,...,...,...,...,...,...
3815978,24173154,cms,cmspilot,1,4,production,0:0,RUNNING,cn461,172800.0,401.0
3815979,24173155,cms,cmspilot,1,4,production,0:0,RUNNING,cn1516,172800.0,244.0
3815980,24173156,cms,cmspilot,1,4,production,0:0,RUNNING,cn1524,172800.0,244.0
3815981,24173158,cms,cmspilot,1,4,production,0:0,RUNNING,cn1532,172800.0,244.0


In [110]:
accre_cms_user_part_30.groupby('STATE')['STATE'].count()

STATE
CANCELLED              1988
CANCELLED by 9201         3
CANCELLED by 9202         3
COMPLETED            471926
FAILED                   61
RUNNING                  63
Name: STATE, dtype: int64

In [100]:
accre_cms_user_part_30_fail = accre_cms_user_part_30[accre_cms_user_part_30['STATE'] == 'FAILED']
accre_cms_user_part_30_fail

,JOBID,ACCOUNT,USER,NODES,CPUS,PARTITION,EXITCODE,STATE,NODELIST,REQTIME_SEC,USEDTIME_SEC
8629,17094238,cms,cmspilot,1,0,nogpfs,1:0,FAILED,cn1554,172800.0,71.0
8632,17094335,cms,cmspilot,1,0,nogpfs,1:0,FAILED,cn1544,172800.0,86.0
22234,17072678,cms,cmspilot,1,8,nogpfs,126:0,FAILED,ng909,172800.0,155.0
197972,17350102,cms,cmspilot,1,32,nogpfs,1:0,FAILED,cn1581,172800.0,92.0
197973,17350123,cms,cmslocal,1,32,nogpfs,1:0,FAILED,cn1563,172800.0,113.0
...,...,...,...,...,...,...,...,...,...,...,...
2194734,20958051,cms,cmslocal,1,4,production,127:0,FAILED,cn1536,172800.0,65.0
2505543,21577870,cms,cmspilot,1,4,production,127:0,FAILED,cn1523,172800.0,115.0
2505544,21577875,cms,cmspilot,1,4,production,127:0,FAILED,cn1523,172800.0,116.0
3259530,23146628,cms,cmspilot,1,8,nogpfs,127:0,FAILED,ng739,172800.0,67.0


In [101]:
accre_cms_user_part_30_fail.describe()

,NODES,CPUS,REQTIME_SEC,USEDTIME_SEC
count,61.0,61.000000,61.0,61.000000
mean,1.0,25.590164,172800.0,80.885246
std,0.0,11.904309,0.0,31.765337
min,1.0,0.000000,172800.0,10.000000
25%,1.0,32.000000,172800.0,68.000000
50%,1.0,32.000000,172800.0,69.000000
75%,1.0,32.000000,172800.0,83.000000
max,1.0,32.000000,172800.0,170.000000


In [113]:
accre_cms_user_part_30_fail.NODELIST.describe()

count         61
unique        52
top       cn1387
freq           3
Name: NODELIST, dtype: object

There were only 61 failed jobs for cms jobs under 30 minutes. There were 52 unique Nodelists, with the most frequent cn1387. 

In [117]:
def find_most_common_values(df,column):
    return df[column].value_counts(ascending=False).iloc[0:50]

In [118]:
find_most_common_values(accre_cms_user_part_30_fail, 'NODELIST')

cn1387    3
cn1563    2
cn1554    2
cn1523    2
cn1582    2
cn1586    2
cn1536    2
cn1581    2
cn1556    1
cn1558    1
cn1564    1
cn1591    1
cn1577    1
cn1551    1
cn1570    1
cn1549    1
cn1277    1
cn1542    1
cn1557    1
cn1589    1
cn1584    1
cn1579    1
cn1555    1
cn1583    1
cn1562    1
cn1576    1
cn1566    1
cn1565    1
cn1569    1
cn1571    1
cn1592    1
ng1068    1
cn1588    1
cn1567    1
cn1575    1
cn1561    1
cn1553    1
cn1590    1
cn1552    1
cn1544    1
cn1450    1
cn1568    1
ng909     1
cn1559    1
ng739     1
cn1572    1
cn1539    1
cn1574    1
cn1550    1
cn1585    1
Name: NODELIST, dtype: int64

Now determine the number of cancelled jobs under 30 minutes

In [102]:
accre_cms_user_part_30_cxl = accre_cms_user_part_30[accre_cms_user_part_30['STATE'] == 'CANCELLED']
accre_cms_user_part_30_cxl

,JOBID,ACCOUNT,USER,NODES,CPUS,PARTITION,EXITCODE,STATE,NODELIST,REQTIME_SEC,USEDTIME_SEC
8400,17094547,cms,cmslocal,1,32,nogpfs,0:0,CANCELLED,cn1559,172800.0,976.0
8401,17094548,cms,cmslocal,1,32,nogpfs,0:0,CANCELLED,cn1573,172800.0,976.0
8402,17094549,cms,cmslocal,1,32,nogpfs,0:0,CANCELLED,cn1574,172800.0,976.0
8403,17094550,cms,cmslocal,1,32,nogpfs,0:0,CANCELLED,cn1575,172800.0,976.0
8404,17094551,cms,cmslocal,1,32,nogpfs,0:0,CANCELLED,cn1577,172800.0,976.0
...,...,...,...,...,...,...,...,...,...,...,...
3801033,24154499,cms,cmspilot,1,4,production,0:0,CANCELLED,cn1488,172800.0,6.0
3811791,24166263,cms,cmspilot,1,1,production,0:0,CANCELLED,cn1276,172800.0,8.0
3814356,24170123,cms,cmslocal,1,4,production,0:0,CANCELLED,cn1202,172800.0,15.0
3814357,24170124,cms,cmslocal,1,4,production,0:0,CANCELLED,cn1204,172800.0,23.0


In [104]:
accre_cms_user_part_30_cxl.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1988 entries, 8400 to 3814434
Data columns (total 11 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   JOBID         1988 non-null   object 
 1   ACCOUNT       1988 non-null   object 
 2   USER          1988 non-null   object 
 3   NODES         1988 non-null   int64  
 4   CPUS          1988 non-null   int64  
 5   PARTITION     1988 non-null   object 
 6   EXITCODE      1988 non-null   object 
 7   STATE         1988 non-null   object 
 8   NODELIST      1988 non-null   object 
 9   REQTIME_SEC   1988 non-null   float64
 10  USEDTIME_SEC  1988 non-null   float64
dtypes: float64(2), int64(2), object(7)
memory usage: 186.4+ KB


In [119]:
# review cancelled jobs under 30 minutes' nodelist.  There are 626 unique nodelists
accre_cms_user_part_30_cxl.NODELIST.describe()

count      1988
unique      626
top       ng518
freq         15
Name: NODELIST, dtype: object

In [81]:
accre_cms_user_part_30.EXITCODE.describe()

count     478641
unique         5
top          0:0
freq      478533
Name: EXITCODE, dtype: object

In [80]:
find_most_common_values(accre_cms_user_part_30, 'NODELIST')

ng518     19345
ng1078     6380
ng1112     6164
ng1040     5673
ng1027     5639
ng1003     5619
ng507      5394
ng1067     5346
ng909      5158
ng205      4686
ng502      4614
ng688      4576
ng1063     4551
ng1020     4544
ng514      4449
ng678      4362
ng915      4359
ng1036     4322
ng1080     4319
ng1014     4254
ng1061     4180
ng734      4121
ng1039     4011
ng733      3984
ng512      3596
ng1068     3474
ng739      3454
ng1079     3163
ng729      2539
ng1033     2505
Name: NODELIST, dtype: int64

In [60]:
accre_cms_user_part['STATE'].describe()

count        835473
unique            6
top       COMPLETED
freq         832505
Name: STATE, dtype: object

KeyError: 'EXITCODE'

In [38]:
# 

KeyError: "None of [Float64Index([171106.0, 171193.0, 171396.0, 170875.0, 170164.0, 171000.0,\n              170718.0, 171331.0, 170157.0, 171307.0,\n              ...\n                  22.0,     23.0,     22.0,     22.0,     22.0,   1136.0,\n                1017.0,   1017.0,    891.0,    990.0],\n             dtype='float64', length=3600)] are in the [columns]"

TypeError: '<' not supported between instances of 'list' and 'int'

In [ ]:
accre_cms.EXITCODE.unique()

In [ ]:
accre.STATE.unique()

In [ ]:
accre.STATE.describe()

In [ ]:
accre.groupby('STATE').size()

In [ ]:
accre_cms.PARTITION.describe()

In [ ]:
accre_cms.PARTITION.unique()

In [ ]:
# review failed nodes, first identify failed jobs
failed = accre[accre['STATE'] == 'FAILED']
failed

In [ ]:
failed.info()

In [ ]:
failed.NODELIST.unique()

In [ ]:
failed.NODELIST.value_counts()

In [ ]:
def find_most_common_values(df,column):
    return df[column].value_counts(ascending=False).iloc[0:30]

In [ ]:
find_most_common_values(failed, 'NODELIST')

In [ ]:
# nodelist = failed['NODELIST'].unique().tolist()
# nodelist_nums = [i.strip('cn').strip('[').strip(']') for i in nodelist]
# for n in nodelist_nums:
#     if ',' in n: 
#         nodes = n.split(',')
#         if '-' in n:
#             nodes = n.split('-')
#             low, high = int(nodes[0]), int(nodes[1])       
#         else:
#             nodes = int(n)


For Question #2, filter to CMS users (cmslocal, cmspilot), examine relationship betw
early (< 60 min) termination jobs, production & nogpfs partitions, and specific nodes
look for commonly failed nodes 

Actions: 
1) pull in code to convert time to seconds 
2) run query for cms users, used time less than 3600 seconds, partition = production, nogpfs.  Assign to cms_1hr
3) analyze exit codes: left = user errors, right = node errors (might be easily identifiable through cms_1hr query)
3) run query for cms_1hr plus State = Failed.  Assign to cms_1hr_fail
4) analyze for commonly occurring failed nodes
 

In [ ]:
accre.head()

In [ ]:
# create query with cms users, jobs less than 1 hour, production & nogpfs partitions


In [ ]:
cms_1hr.info()

In [ ]:
cms_1hr.USER.unique()

In [ ]:
#  & (accre["USEDTIME"] < '01:00')  should I use brackets instead?  [accre_cms['USEDTIME'] < '1.00']

In [ ]:
#pd.crosstab(accre_cms['column'], accre_cms['column'], normalize='index')

In [ ]:
#accre_cms = (accre_cms["ACCOUNT"] == 'cms') & (accre_["USER"] == 'cmslocal') | (accre["USER"] == 'cmspilot')

In [ ]:
# accre_cms = accre[['JOBID', 'ACCOUNT', 'USER', 'REQTIME', 'USEDTIME', 'PARTITION', 'EXITCODE', 'STATE', 'NODELIST']]
# accre_cms

In [ ]:
#accre_cms_1hr = accre_cms_user_part(['USEDTIME_SEC'] < 3600)

In [ ]:
#accre_cms_state_piv = (accre_cms_user_part[accre_cms_user_part.STATE.isin(['CANCELLED', 'FAILED'])]
#                 .pivot_table(values = accre_cms_user_part[accre_cms_user_part['USEDTIME_SEC'][0:3600]], columns = 'STATE', index = accre_cms_user_part['NODELIST'])
#                 .dropna()
#                )

In [ ]:
#accre_cms_user_part_30.groupby('EXITCODE')['NODELIST']

In [ ]:
#failed_part = accre[(accre["STATE"] == 'FAILED') & (accre['PARTITION'] == 'production')]
#failed.head() 